In [8]:
from rdkit import Chem

In [9]:
LIST_SYMBOLS = ['C', 'N', 'O', 'S', 'F', 'H', 'Si', 'P', 'Cl', 'Br',
            'Li', 'Na', 'K', 'Mg', 'Ca', 'Fe', 'As', 'Al', 'I', 'B',
            'V', 'Tl', 'Sb', 'Sn', 'Ag', 'Pd', 'Co', 'Se', 'Ti', 'Zn',
            'Ge', 'Cu', 'Au', 'Ni', 'Cd', 'Mn', 'Cr', 'Pt', 'Hg', 'Pb']


def atom_feature(atom):
    return np.array(char_to_ix(atom.GetSymbol(), LIST_SYMBOLS) +
                    char_to_ix(atom.GetDegree(), [0, 1, 2, 3, 4, 5]) +
                    char_to_ix(atom.GetTotalNumHs(), [0, 1, 2, 3, 4]) +
                    char_to_ix(atom.GetImplicitValence(), [0, 1, 2, 3, 4, 5]) +
                    char_to_ix(int(atom.GetIsAromatic()), [0, 1]))    # (40, 6, 5, 6, 2)


def char_to_ix(x, allowable_set):
    if x not in allowable_set:
        return [0] # Unknown Atom Token
    return [allowable_set.index(x)+1]


def mol2graph(smi, MAX_LEN):
    mol = Chem.MolFromSmiles(smi)

    X = np.zeros((MAX_LEN, 5), dtype=np.uint8)
    A = np.zeros((MAX_LEN, MAX_LEN), dtype=np.uint8)

    temp_A = Chem.rdmolops.GetAdjacencyMatrix(mol).astype(np.uint8, copy=False)[:MAX_LEN, :MAX_LEN]
    num_atom = temp_A.shape[0]
    A[:num_atom, :num_atom] = temp_A + np.eye(temp_A.shape[0], dtype=np.uint8)
    
    for i, atom in enumerate(mol.GetAtoms()):
        feature = atom_feature(atom)
        X[i, :] = feature
        if i + 1 >= num_atom: break
            
    return X, A

smiles = "O=C(NCc1ccccn1)c2ccc(Oc3ccccc3)cc2"
X, A = mol2graph(smiles, 70)

In [10]:
X, A

(array([[3, 2, 1, 1, 1],
        [1, 4, 1, 1, 1],
        [2, 3, 2, 2, 1],
        [1, 3, 3, 3, 1],
        [1, 4, 1, 1, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [2, 3, 1, 1, 2],
        [1, 4, 1, 1, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 4, 1, 1, 2],
        [3, 3, 1, 1, 1],
        [1, 4, 1, 1, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [1, 3, 2, 2, 2],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],


In [ ]:
import numpy as np

def calculate_score(y_true, y_pred):
    # IC50(nM) to pIC50 변환
    def to_pIC50(IC50):
        return -np.log10(IC50 * 1e-9)
    
    y_true_pIC50 = to_pIC50(y_true)
    y_pred_pIC50 = to_pIC50(y_pred)
    
    # Normalized RMSE 계산
    rmse = np.sqrt(np.mean((y_true - y_pred)**2))
    normalized_rmse = rmse / (np.max(y_true) - np.min(y_true))
    
    # Correct Ratio 계산
    absolute_errors_pIC50 = np.abs(y_true_pIC50 - y_pred_pIC50)
    correct_ratio = np.mean(absolute_errors_pIC50 <= 0.5)
    
    # 최종 점수 계산
    A = normalized_rmse
    B = correct_ratio
    score = 0.5 * (1 - min(A, 1)) + 0.5 * B
    
    return score, normalized_rmse, correct_ratio

# 사용 예시
y_true = np.array([100, 200, 300, 400, 500])  # 실제 IC50(nM) 값
y_pred = np.array([110, 190, 280, 420, 510])  # 예측된 IC50(nM) 값

final_score, normalized_rmse, correct_ratio = calculate_score(y_true, y_pred)

print(f"Normalized RMSE: {normalized_rmse:.4f}")
print(f"Correct Ratio: {correct_ratio:.4f}")
print(f"Final Score: {final_score:.4f}")

# Submission

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim


class BN1d(nn.Module):
    def __init__(self, out_dim, use_bn):
        super(BN1d, self).__init__()
        self.use_bn = use_bn
        self.bn = nn.BatchNorm1d(out_dim)
             
    def forward(self, x):
        if not self.use_bn:
            return  x
        origin_shape = x.shape
        x = x.view(-1, origin_shape[-1])
        x = self.bn(x)
        x = x.view(origin_shape)
        return x

    
class GConv(nn.Module):
    def __init__(self, input_dim, output_dim, use_bn):
        super(GConv, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        self.bn = BN1d(output_dim, use_bn)
        self.relu = nn.ReLU()
        
    def forward(self, X, A):
        x = self.fc(X)
        x = torch.matmul(A, x)
        x = self.relu(self.bn(x))
        return x, A
        
    
class Readout(nn.Module):
    def __init__(self, out_dim, molvec_dim):
        super(Readout, self).__init__()
        self.readout_fc = nn.Linear(out_dim, molvec_dim)
        nn.init.xavier_normal_(self.readout_fc.weight.data)

    def forward(self, output_H):
        molvec = self.readout_fc(output_H)
        molvec = torch.mean(molvec, dim=1)
        return molvec
    

class GCNNet(nn.Module):
    
    def __init__(self, args):
        super(GCNNet, self).__init__()
        
        # Create Atom Element embedding layer
        self.embedding = self.create_emb_layer([args.vocab_size, args.degree_size,
                                                args.numH_size, args.valence_size,
                                                args.isarom_size],  args.emb_train)    
        
        self.gcn_layers = nn.ModuleList()
        for i in range(args.n_layer):
            self.gcn_layers.append(GConv(args.in_dim if i==0 else args.out_dim, args.out_dim, args.use_bn))
                                   
        self.readout = Readout(args.out_dim, args.molvec_dim)
        
        self.fc1 = nn.Linear(args.molvec_dim, args.molvec_dim//2)
        self.fc2 = nn.Linear(args.molvec_dim//2, args.molvec_dim//2)
        self.fc3 = nn.Linear(args.molvec_dim//2, 1)
        self.relu = nn.ReLU()
        
    def create_emb_layer(self, list_vocab_size, emb_train=False):
        list_emb_layer = nn.ModuleList()
        for i, vocab_size in enumerate(list_vocab_size):
            vocab_size += 1
            emb_layer = nn.Embedding(vocab_size, vocab_size)
            weight_matrix = torch.zeros((vocab_size, vocab_size))
            for i in range(vocab_size):
                weight_matrix[i][i] = 1
            emb_layer.load_state_dict({'weight': weight_matrix})
            emb_layer.weight.requires_grad = emb_train
            list_emb_layer.append(emb_layer)
        return list_emb_layer

    def _embed(self, x):
        list_embed = list()
        for i in range(5):
            list_embed.append(self.embedding[i](x[:, :, i]))
        x = torch.cat(list_embed, 2)
        return x
        
    def forward(self, x, A):
        A = A.float()
        x = self._embed(x)   
        
        for i, module in enumerate(self.gcn_layers):
            x, A = module(x, A)
        x = self.readout(x)
        
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return torch.squeeze(x)
        

In [47]:
# load model
import torch
import argparse

parser = argparse.ArgumentParser()
args = parser.parse_args("")

# ==== Embedding Config ==== #
args.max_len = 70
args.vocab_size = 40
args.degree_size = 6
args.numH_size = 5
args.valence_size = 6
args.isarom_size = 2
args.emb_train = True


# ==== Model Architecture Config ==== #
args.in_dim = 64
args.out_dim = 256
args.molvec_dim = 512
args.n_layer = 1
args.use_bn = True
args.act = 'relu'
args.dp_rate = 0.3
args.n_layer = 3


# ==== Optimizer Config ==== #
args.lr = 0.005
args.l2_coef = 0.0001
args.optim = 'ADAM'


# ==== Training Config ==== #
args.epoch = 300
args.batch_size = 256
args.device = 'cpu'
args.exp_name = 'exp1_lr_stage'

model = GCNNet(args)
model.load_state_dict(torch.load('./gnn/model/exp1_lr_stage_6.pt'))
model.eval()

GCNNet(
  (embedding): ModuleList(
    (0): Embedding(41, 41)
    (1): Embedding(7, 7)
    (2): Embedding(6, 6)
    (3): Embedding(7, 7)
    (4): Embedding(3, 3)
  )
  (gcn_layers): ModuleList(
    (0): GConv(
      (fc): Linear(in_features=64, out_features=256, bias=True)
      (bn): BN1d(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
    (1): GConv(
      (fc): Linear(in_features=256, out_features=256, bias=True)
      (bn): BN1d(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
    (2): GConv(
      (fc): Linear(in_features=256, out_features=256, bias=True)
      (bn): BN1d(
        (bn): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (relu): ReLU()
    )
  )
  (readout): Readout(
    (readout_fc): Linear(in_features=256, out_features=512, bias=True)
  )
  (fc1): Linear(in_

In [48]:
class gcnDataset(Dataset):
    def __init__(self, df, max_len=120):
        self.smiles = df["Smiles"]
        # self.exp = df["pIC50"].values
                
        list_X = list()
        list_A = list()
        for i, smiles in enumerate(self.smiles):
            X, A = mol2graph(smiles, max_len)
            list_X.append(X)
            list_A.append(A)
            
        self.X = np.array(list_X, dtype=np.uint8)
        self.A = np.array(list_A, dtype=np.uint8)
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.A[index]#, self.exp[index]

In [49]:
test_dataset = gcnDataset(pd.read_csv('./data/test.csv'))

In [50]:
def pIC50_to_IC50(pic50_values):
    """Convert pIC50 values to IC50 (nM)."""
    return 10 ** (9 - pic50_values)

In [52]:
X = torch.tensor(test_dataset.X).long().to(args.device).long()
A = torch.tensor(test_dataset.A).long().to(args.device).long()
test_y_pred = model.forward(X, A).cpu().detach().numpy().reshape(-1)
test_y_pred

array([6.2238646, 7.612804 , 7.3333554, 7.151793 , 6.9687276, 7.137066 ,
       7.0495553, 6.894513 , 7.600179 , 6.656602 , 6.907613 , 7.3785477,
       6.966046 , 6.9216743, 7.826873 , 7.259881 , 7.816792 , 6.8753734,
       7.033349 , 7.2657256, 7.32243  , 7.497427 , 6.99265  , 7.2713027,
       6.745974 , 6.9128375, 7.203873 , 7.62736  , 7.227302 , 7.180169 ,
       7.070168 , 7.0036283, 7.0334888, 6.709858 , 7.9083276, 7.0249667,
       6.7294645, 7.56149  , 7.144639 , 7.1783876, 7.0142813, 7.1453905,
       6.7086077, 7.360337 , 7.9244056, 7.1918197, 7.277575 , 6.8588295,
       6.867202 , 7.0831056, 7.4431014, 7.5847282, 7.291629 , 6.8805604,
       7.422931 , 7.179434 , 6.8091846, 6.877061 , 7.8730574, 7.8708124,
       7.0804563, 6.8893094, 7.050582 , 7.829168 , 8.064034 , 7.085113 ,
       7.496227 , 7.491429 , 6.9045005, 7.6083245, 7.2354274, 7.0089593,
       7.018497 , 7.117084 , 6.9912   , 6.813895 , 7.095908 , 7.0787406,
       7.2260995, 7.7742963, 7.4507174, 7.5950084, 

In [55]:
submit = pd.DataFrame(columns=['ID', 'IC50_nM'])
submit['IC50_nM'] = pIC50_to_IC50(test_y_pred)
submit['ID'] = pd.read_csv('./data/test.csv').ID
submit.head()

submit.to_csv('./data/gcn_submit.csv', index=False)